# Mass Data Query (all metadata IDs)


#### **ONLY RUN IF MASSIVE AMOUNTS OF DATA IS YOUR GOAL**

This notebook runs a query for each groupMetadataID in a csv file, retrieving chassis and pose data.

It could likely be sped up by querying on once for all pose and chassis data, then breaking down by each ID and topic.


In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import os

# Import plotly.express to utilize map

### Import your groupMetadataIDs below and choose to query data or not


In [2]:
from boto3.dynamodb.conditions import Key, Attr
import boto3

# Get the service resource
dynamodb = boto3.resource('dynamodb')
# Instantiate table resource objects
table = dynamodb.Table('ads_passenger_processed')
meta_table = dynamodb.Table('ads_passenger_processed_metadata')

### Query Best_pose data


In [3]:
# query arguments for best_pose data
keywords = dict(
    IndexName='topic-index',
    KeyConditionExpression=Key('topic').eq(
        '/apollo/sensor/gnss/best_pose'),
    ProjectionExpression="groupMetadataID, #t, solStatus, solType, latitudeStdDev, longitudeStdDev, numSatsTracked, numSatsMulti, numSatsInSolution, differentialAge, solutionAge, latitude, longitude",

    Limit=1500,
    ExpressionAttributeNames={'#t': 'time'},
)

# run initial query and coerce results into a dataframe
res = table.query(**keywords)
df_pose = pd.DataFrame.from_dict(
    pd.json_normalize(res['Items']), orient='columns')

# track evaluated last key to get new data
done = False
leek = res["LastEvaluatedKey"]

while not done:
    try:
        # repeat query until we've exhausted the dataset
        leek = res["LastEvaluatedKey"]
        keywords["ExclusiveStartKey"] = leek
        res = table.query(**keywords)
        tmpDF = pd.DataFrame.from_dict(
            pd.json_normalize(res['Items']), orient='columns')
        df_pose = pd.concat([df_pose, tmpDF])
    except KeyError:
        print(f"done querying best_pose data")
        done = True

done querying best_pose data


### Query Chassis data


Warning: This query can take awhile. 2+ hours at 48 groupMetadataIDs


In [4]:
# query arguments for chassis data
keywords = dict(
    IndexName='topic-index',
    KeyConditionExpression=Key('topic').eq(
        '/apollo/canbus/chassis'),
    Limit=2000,
)

# run initial query and coerce results into a dataframe
res = table.query(**keywords)
df_chassis = pd.DataFrame.from_dict(
    pd.json_normalize(res['Items']), orient='columns')

# track evaluated last key to get new data
done = False
leek = res["LastEvaluatedKey"]

while not done:
    try:
        # repeat query until we've exhausted the dataset
        leek = res["LastEvaluatedKey"]
        keywords["ExclusiveStartKey"] = leek
        res = table.query(**keywords)
        tmpDF = pd.DataFrame.from_dict(
            pd.json_normalize(res['Items']), orient='columns')
        df_chassis = pd.concat([df_chassis, tmpDF])

    except KeyError:
        print(f"done querying chassis data")
        done = True

done querying chassis data


In [5]:
print("\t\t\t(rows, columns)")
print("df_pose dimensions:\t", df_pose.shape)
print("df_chassis dimensions:\t", df_chassis.shape)

			(rows, columns)
df_pose dimensions:	 (90086, 13)
df_chassis dimensions:	 (3823886, 31)


In [6]:
temp_pose = df_pose.copy()
temp_chassis = df_chassis.copy()

In [7]:
ChassisGmIDs = sorted(temp_chassis["groupMetadataID"].unique())
ChassisGmIDs

['06cbdbc0-db4d-11ee-a158-97f8443fd730',
 '072ef896-cbac-11ee-909c-e1dc60cf66f9',
 '154fab12-a43f-11ee-88ec-eb6a8d5269b4',
 '19b7ebd0-d9b7-11ee-a158-97f8443fd730',
 '1bbbfbae-c839-11ee-a7fc-dd032dba19e8',
 '25641404-cb66-11ee-909c-e1dc60cf66f9',
 '271fee10-cb8b-11ee-909c-e1dc60cf66f9',
 '2837eb9c-9542-11ee-956e-9da2d070324c',
 '286c70cc-d2f7-11ee-b437-336917683bb8',
 '2bc6ebb8-a529-11ee-88ec-eb6a8d5269b4',
 '39ba7438-d0d5-11ee-9435-f7e542e2436c',
 '3a116996-93a9-11ee-956e-9da2d070324c',
 '3a2a78cc-db21-11ee-a158-97f8443fd730',
 '3c415ade-d353-11ee-b437-336917683bb8',
 '3d8020aa-cb7f-11ee-909c-e1dc60cf66f9',
 '47561998-d9c3-11ee-a158-97f8443fd730',
 '51ef6da6-ca9f-11ee-909c-e1dc60cf66f9',
 '559495ca-d270-11ee-b437-336917683bb8',
 '58263e34-a45c-11ee-88ec-eb6a8d5269b4',
 '5976b77a-a504-11ee-88ec-eb6a8d5269b4',
 '64737d98-d312-11ee-b437-336917683bb8',
 '68c289fa-dbd4-11ee-a158-97f8443fd730',
 '69ab88ec-dc17-11ee-a158-97f8443fd730',
 '6af236d6-d98f-11ee-a158-97f8443fd730',
 '6d2ea45a-c839-

In [8]:
PoseGmIDs = sorted(temp_pose["groupMetadataID"].unique())
PoseGmIDs

['06cbdbc0-db4d-11ee-a158-97f8443fd730',
 '072ef896-cbac-11ee-909c-e1dc60cf66f9',
 '154fab12-a43f-11ee-88ec-eb6a8d5269b4',
 '19b7ebd0-d9b7-11ee-a158-97f8443fd730',
 '1bbbfbae-c839-11ee-a7fc-dd032dba19e8',
 '25641404-cb66-11ee-909c-e1dc60cf66f9',
 '271fee10-cb8b-11ee-909c-e1dc60cf66f9',
 '2837eb9c-9542-11ee-956e-9da2d070324c',
 '286c70cc-d2f7-11ee-b437-336917683bb8',
 '2bc6ebb8-a529-11ee-88ec-eb6a8d5269b4',
 '39ba7438-d0d5-11ee-9435-f7e542e2436c',
 '3a116996-93a9-11ee-956e-9da2d070324c',
 '3a2a78cc-db21-11ee-a158-97f8443fd730',
 '3c415ade-d353-11ee-b437-336917683bb8',
 '3d8020aa-cb7f-11ee-909c-e1dc60cf66f9',
 '47561998-d9c3-11ee-a158-97f8443fd730',
 '51ef6da6-ca9f-11ee-909c-e1dc60cf66f9',
 '559495ca-d270-11ee-b437-336917683bb8',
 '58263e34-a45c-11ee-88ec-eb6a8d5269b4',
 '5976b77a-a504-11ee-88ec-eb6a8d5269b4',
 '64737d98-d312-11ee-b437-336917683bb8',
 '68c289fa-dbd4-11ee-a158-97f8443fd730',
 '69ab88ec-dc17-11ee-a158-97f8443fd730',
 '6af236d6-d98f-11ee-a158-97f8443fd730',
 '6d2ea45a-c839-

In [9]:
if len(ChassisGmIDs) == len(PoseGmIDs) and (np.array(ChassisGmIDs) == np.array(PoseGmIDs)).all():
    print("all gmIDs match")
    gmIDs = ChassisGmIDs
else:
    print("some different gmIDs")

all gmIDs match


### Query Metadata


In [10]:
# query for the bestPosData
df_meta = pd.DataFrame()

for id in ChassisGmIDs:

    res = meta_table.query(
        IndexName='groupMetadataID-index',
        ProjectionExpression="groupMetadataID, #t, driver, #i, experimentID, #o.Weather, #o.#m, #o.Notes",
        KeyConditionExpression=Key('groupMetadataID').eq(id),
        Limit=1,
        ExpressionAttributeNames={'#t': 'time', "#i": '_id',
                                  "#o": "other", "#m": "Map"},

    )
    tempDF = pd.DataFrame.from_dict(
        pd.json_normalize(res['Items']), orient='columns')

    df_meta = pd.concat([df_meta, tempDF])

print("df_meta dimensions:\t", df_meta.shape)
temp_meta = df_meta.copy()

df_meta dimensions:	 (59, 8)


In [11]:
MetaGmIDs = sorted(temp_meta["groupMetadataID"].unique())
MetaGmIDs

['06cbdbc0-db4d-11ee-a158-97f8443fd730',
 '072ef896-cbac-11ee-909c-e1dc60cf66f9',
 '154fab12-a43f-11ee-88ec-eb6a8d5269b4',
 '19b7ebd0-d9b7-11ee-a158-97f8443fd730',
 '1bbbfbae-c839-11ee-a7fc-dd032dba19e8',
 '25641404-cb66-11ee-909c-e1dc60cf66f9',
 '271fee10-cb8b-11ee-909c-e1dc60cf66f9',
 '2837eb9c-9542-11ee-956e-9da2d070324c',
 '286c70cc-d2f7-11ee-b437-336917683bb8',
 '2bc6ebb8-a529-11ee-88ec-eb6a8d5269b4',
 '39ba7438-d0d5-11ee-9435-f7e542e2436c',
 '3a116996-93a9-11ee-956e-9da2d070324c',
 '3a2a78cc-db21-11ee-a158-97f8443fd730',
 '3c415ade-d353-11ee-b437-336917683bb8',
 '3d8020aa-cb7f-11ee-909c-e1dc60cf66f9',
 '47561998-d9c3-11ee-a158-97f8443fd730',
 '51ef6da6-ca9f-11ee-909c-e1dc60cf66f9',
 '559495ca-d270-11ee-b437-336917683bb8',
 '58263e34-a45c-11ee-88ec-eb6a8d5269b4',
 '5976b77a-a504-11ee-88ec-eb6a8d5269b4',
 '64737d98-d312-11ee-b437-336917683bb8',
 '68c289fa-dbd4-11ee-a158-97f8443fd730',
 '69ab88ec-dc17-11ee-a158-97f8443fd730',
 '6af236d6-d98f-11ee-a158-97f8443fd730',
 '6d2ea45a-c839-

In [12]:
if not os.path.exists(f"./data/best_pose"):
    os.makedirs(f"./data/best_pose")

if not os.path.exists(f"./data/chassis"):
    os.makedirs(f"./data/chassis")

if not os.path.exists("./data/metadata"):
    os.makedirs("./data/metadata")

i = 0

for id in sorted(ChassisGmIDs):
    print(f"{i}: {id}")
    temp = temp_chassis[temp_chassis["groupMetadataID"] == id]
    temp.to_csv(f"./data/chassis/{id}.csv", index=False)

    temp = temp_pose[temp_pose["groupMetadataID"] == id]
    temp.to_csv(f"./data/best_pose/{id}.csv", index=False)

    temp = temp_meta[temp_meta["groupMetadataID"] == id]
    temp.to_csv(f"./data/metadata/{id}.csv", index=False)

print("complete")

06cbdbc0-db4d-11ee-a158-97f8443fd730
072ef896-cbac-11ee-909c-e1dc60cf66f9
154fab12-a43f-11ee-88ec-eb6a8d5269b4
19b7ebd0-d9b7-11ee-a158-97f8443fd730
1bbbfbae-c839-11ee-a7fc-dd032dba19e8
25641404-cb66-11ee-909c-e1dc60cf66f9
271fee10-cb8b-11ee-909c-e1dc60cf66f9
2837eb9c-9542-11ee-956e-9da2d070324c
286c70cc-d2f7-11ee-b437-336917683bb8
2bc6ebb8-a529-11ee-88ec-eb6a8d5269b4
39ba7438-d0d5-11ee-9435-f7e542e2436c
3a116996-93a9-11ee-956e-9da2d070324c
3a2a78cc-db21-11ee-a158-97f8443fd730
3c415ade-d353-11ee-b437-336917683bb8
3d8020aa-cb7f-11ee-909c-e1dc60cf66f9
47561998-d9c3-11ee-a158-97f8443fd730
51ef6da6-ca9f-11ee-909c-e1dc60cf66f9
559495ca-d270-11ee-b437-336917683bb8
58263e34-a45c-11ee-88ec-eb6a8d5269b4
5976b77a-a504-11ee-88ec-eb6a8d5269b4
64737d98-d312-11ee-b437-336917683bb8
68c289fa-dbd4-11ee-a158-97f8443fd730
69ab88ec-dc17-11ee-a158-97f8443fd730
6af236d6-d98f-11ee-a158-97f8443fd730
6d2ea45a-c839-11ee-a7fc-dd032dba19e8
76683d3c-db18-11ee-a158-97f8443fd730
787d9684-d2c2-11ee-b437-336917683bb8
7